In [10]:
import os
import re
import nest_asyncio
nest_asyncio.apply()

from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader, Document

In [2]:
LLAMAPARSE_API_KEY = os.environ.get('LLAMAPARSE_API_KEY')
if LLAMAPARSE_API_KEY is not None:
    print('API key found')
else:
    print('Check for API key in environment variable')

API key found


In [8]:
# instantiate parser
parser = LlamaParse(
    api_key=LLAMAPARSE_API_KEY,
    result_type="markdown", # or text
    num_workers=4, # for multiple files
    verbose=True,
    language="en", # default is english
)

In [6]:
file_list = [file for file in os.listdir('../data') if file.startswith('ufr')]
file_list


['ufr-q1fy24.pdf', 'ufr-q3fy24.pdf', 'ufr-q2fy24.pdf']

In [22]:
file_extractor = {".pdf": parser}
filename_fn = lambda filename: {"file_name": filename}

In [23]:

pattern = r"(?<=-)\w+(?=\.)"

re_dash_to_dot = re.compile(r"-([^\.]+)\.")
print(re.search(re_dash_to_dot, file_list[0]))
print(re.search(pattern, file_list[0]))
re.search(pattern, file_list[0]).group()

<re.Match object; span=(3, 11), match='-q1fy24.'>
<re.Match object; span=(4, 10), match='q1fy24'>


'q1fy24'

In [35]:
doc_list = {}

for file in file_list:
    reader = SimpleDirectoryReader(
        input_files=['../data/' + file],
        file_extractor=file_extractor,
        # filename_as_id=True,
        file_metadata=filename_fn,
        )
    doc = reader.load_data()
    for i in doc:
        i.metadata['file_descr']=f'Axis bank quarterly earnings report for {re.search(pattern, file).group()} '
    doc_list[file] = doc

Started parsing the file under job_id 8180bb39-ee30-4f03-a342-324bfefb79b1
Started parsing the file under job_id 29c7c45e-cb94-4127-991b-b44f044cf0dd
Started parsing the file under job_id 7a680565-3448-4dd7-8906-a8f4a42b5b95


In [40]:
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import MarkdownElementNodeParser

In [42]:
llm = OpenAI(model='gpt-3.5-turbo-0125', temperature=0.1)
node_parser=MarkdownElementNodeParser(llm=llm)

In [ ]:
nodes = node_parser.get_nodes_from_documents(documents)
base_nodes, objects = noder_parser.get_nodes_and_objects(nodes)

In [5]:
from llama_index.postprocessor.flag_embedding_reranker import (
    FlagEmbeddingReranker,
    )

ModuleNotFoundError: No module named 'llama_index.postprocessor'

In [2]:
from llama_index.postprocessor.llama_index_postprocessor_flag_embedding_reranker import (
    FlagEmbeddingReranker,
)

rerank = FlagEmbeddingReranker(model="BAAI/bge-reranker-large", top_n=5)

ModuleNotFoundError: No module named 'llama_index.postprocessor'